In [57]:
import chromadb
from chromadb.utils import embedding_functions

In [4]:
chroma_client = chromadb.Client()
daf_ef = embedding_functions.DefaultEmbeddingFunction()


In [55]:
namee = daf_ef("shekar")
print(namee)

[array([-8.81500840e-02,  3.89120951e-02, -6.26745373e-02,  2.59769447e-02,
        2.12722141e-02,  3.61654274e-02,  6.47250935e-02,  3.27302478e-02,
       -2.28122417e-02,  3.42626460e-02, -1.14768185e-02, -5.58527336e-02,
        7.52527118e-02,  2.89201047e-02, -1.21848667e-02, -6.14353009e-02,
        5.79327904e-02, -2.35474817e-02, -3.74577716e-02,  7.78331235e-03,
       -4.38943878e-02, -5.84626861e-04, -5.00517823e-02,  5.25633246e-02,
        4.10172381e-02,  2.72476766e-02, -7.76917441e-03,  6.66312734e-03,
       -5.82849495e-02, -5.82765006e-02, -8.28325748e-03,  1.15402667e-02,
        9.88361910e-02,  1.42464601e-02,  2.19768882e-02, -4.24418710e-02,
        1.69866383e-02,  5.45906499e-02,  2.77194884e-02,  4.03754711e-02,
       -7.19257370e-02, -6.97317123e-02, -7.51370611e-03,  4.05736379e-02,
        3.17095928e-02,  2.00856645e-02, -2.44559105e-02,  2.17489935e-02,
        2.16655061e-02, -7.82397911e-02,  1.23016452e-02, -7.26290345e-02,
        2.04674318e-03, 

In [5]:
collection_name = 'test_collection'

In [6]:
collection = chroma_client.get_or_create_collection(
    collection_name, embedding_function=daf_ef
)

In [7]:
# Define text documents
documents = [
    {"id": "doc1", "text": "Hello, world!"},
    {"id": "doc2", "text": "How are you today?"},
    {"id": "doc3", "text": "Goodbye, see you later!"},
]

In [38]:
type(documents), documents[0]

(list, {'id': 'doc1', 'text': 'Hello, world!'})

In [9]:
for doc in documents:
    collection.upsert(ids = doc["id"], documents=[doc["text"]])

In [28]:
query = "hi world"

In [36]:
res = collection.query(
    query_texts=[query],
    n_results=3
)
res

{'ids': [['doc1', 'doc2', 'doc3']],
 'embeddings': None,
 'documents': [['Hello, world!',
   'How are you today?',
   'Goodbye, see you later!']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None, None]],
 'distances': [[0.5336990356445312, 1.3996727466583252, 1.5729403495788574]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [35]:
res.values()

dict_values([[['doc1', 'doc2']], None, [['Hello, world!', 'How are you today?']], None, None, [[None, None]], [[0.5336990356445312, 1.3996727466583252]], [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]])

In [46]:
res["documents"][0]

['Hello, world!', 'How are you today?', 'Goodbye, see you later!']

In [54]:
type(res["ids"]),res["ids"][0]

(list, ['doc1', 'doc2', 'doc3'])

In [51]:
for idx,doc in enumerate(res["documents"][0]):
    doc_id = res["ids"][0][idx],
    distance = res["distances"][0][idx] 
    print(
        f" For the query: {query}, \n Found similar document: {doc} (ID: {doc_id}, Distance: {distance})"
    )
    

 For the query: hi world, 
 Found similar document: Hello, world! (ID: ('doc1',), Distance: 0.5336990356445312)
 For the query: hi world, 
 Found similar document: How are you today? (ID: ('doc2',), Distance: 1.3996727466583252)
 For the query: hi world, 
 Found similar document: Goodbye, see you later! (ID: ('doc3',), Distance: 1.5729403495788574)


## Integrating VectorDB with LLM(OpenAI)

In [ ]:
import os
from dotenv import load_dotenv
import chromadb
from openai import OpenAI
from chromadb.utils import embedding_functions

# Load environment variables from .env file
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_key, model_name="text-embedding-3-small"
)


# Initialize the Chroma client with persistence
chroma_client = chromadb.PersistentClient(path="./db/chroma_persistent_storage")
collection_name = "document_qa_collection"
collection = chroma_client.get_or_create_collection(
    name=collection_name, embedding_function=openai_ef
)

client = OpenAI(api_key=openai_key)


# =================================
# === For initial setup -- Uncomment (below) all for the first run, and then comment it all out ===
# =================================
# Function to load documents from a directory
def load_documents_from_directory(directory_path):
    print("==== Loading documents from directory ====")
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            with open(
                os.path.join(directory_path, filename), "r", encoding="utf-8"
            ) as file:
                documents.append({"id": filename, "text": file.read()})
    return documents


# Function to split text into chunks
def split_text(text, chunk_size=1000, chunk_overlap=20):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - chunk_overlap
    return chunks


# Load documents from the directory
# directory_path = "./data/new_articles"
directory_path = "C:\\Users\\somashekar.inguva\\OneDrive - Telstra\\Documents\\GenAI_Projects\\vector-databases-course\\data\\new_articles"
documents = load_documents_from_directory(directory_path)

# Split the documents into chunks
chunked_documents = []
for doc in documents:
    chunks = split_text(doc["text"])
    print("==== Splitting docs into chunks ====")
    for i, chunk in enumerate(chunks):
        chunked_documents.append({"id": f"{doc['id']}_chunk{i+1}", "text": chunk})


# Function to generate embeddings using OpenAI API
def get_openai_embedding(text):
    response = client.embeddings.create(input=text, model="text-embedding-3-small")
    embedding = response.data[0].embedding
    print("==== Generating embeddings... ====")
    return embedding


# Generate embeddings for the document chunks
for doc in chunked_documents:
    print("==== Generating embeddings... ====")
    doc["embedding"] = get_openai_embedding(doc["text"])


# Upsert documents with embeddings into Chroma
for doc in chunked_documents:
    print("==== Inserting chunks into db;;; ====")
    collection.upsert(
        ids=[doc["id"]], documents=[doc["text"]], embeddings=[doc["embedding"]]
    )


# === End of the initial setup -- Uncomment all for the first run, and then comment it all out ===
# =================================


# Function to query documents
# def query_documents(question, n_results=2):
#     # query_embedding = get_openai_embedding(question)
#     results = collection.query(query_texts=question, n_results=n_results)

#     # Extract the relevant chunks
#     relevant_chunks = [doc for sublist in results["documents"] for doc in sublist]
#     print("==== Returning relevant chunks ====")

==== Loading documents from directory ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== 

In [59]:
def query_documents(question, n_results=2):
    # query_embedding = get_openai_embedding(question)
    results = collection.query(query_texts=question, n_results=n_results)

    # Extract the relevant chunks
    relevant_chunks = [doc for sublist in results["documents"] for doc in sublist]
    print("==== Returning relevant chunks ====")
    return relevant_chunks

In [62]:
question = "give me a brief overview of the articles about Databricks and mentioned about Cloudera. Be concise, please."
relevant_chunks = query_documents(question)
# answer = generate_response(question, relevant_chunks)
print(relevant_chunks)

==== Returning relevant chunks ====
['l be able to use to provide solutions to their customers.\n\nWith this acquisition, Databricks is also bringing Okera co-founder and CEO Nong Li on board. Li created the Apache Parquet data storage format and was actually briefly an engineer at Databricks between working at Cloudera and before starting Okera, where he was the founding CTO and became the CEO in February 2022.\n\n“As data continues to grow in volume, velocity, and variety across different applications, CIOs, CDOs, and CEOs across the board have to balance those two often conflicting initiatives – not to mention that historically, managing access policies across multiple clouds has been painful and time-consuming,” writes Li in today’s announcement. “Many organizations don’t have enough technical talent to manage access policies at scale, especially with the explosion of LLMs. What they need is a modern, AI-centric governance solution. We could not be more excited to join the Databric

In [63]:
# Function to generate a response from OpenAI
def generate_response(question, relevant_chunks):
    context = "\n\n".join(relevant_chunks)
    prompt = (
        "You are an assistant for question-answering tasks. Use the following pieces of "
        "retrieved context to answer the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the answer concise."
        "\n\nContext:\n" + context + "\n\nQuestion:\n" + question
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
            {
                "role": "user",
                "content": question,
            },
        ],
    )

    answer = response.choices[0].message
    return answer


In [65]:
question = "give me a brief overview of the articles about Databricks and mentioned about Cloudera. Be concise, please."
relevant_chunks = query_documents(question)
answer = generate_response(question, relevant_chunks)
# print(relevant_chunks)
print("==== Answer ====")
print(answer.content)

==== Returning relevant chunks ====
==== Answer ====
The article discusses Databricks acquiring Okera, which was co-founded by Nong Li, who was an engineer at Databricks and worked at Cloudera before starting Okera. Nong Li created the Apache Parquet data storage format and emphasized the need for modern AI-centric governance solutions in managing access policies across multiple clouds. Databricks plans to integrate Okera's technology into its Unity Catalog for enhanced data governance.


### SelfNotes:
Great question — and it's subtle, so let's dive in carefully.

You're absolutely right that **ChromaDB** (or any vector database) **already retrieves relevant results** based on cosine similarity of embeddings. In that sense, *ChromaDB already gives you the closest matches* to a query.

So why pass the ChromaDB results to OpenAI afterward?  
Here’s the key:

| ChromaDB | OpenAI (LLM) |
|:---|:---|
| Finds the most similar *documents/fragments* based on vector space math (cosine similarity). | Synthesizes, summarizes, reasons, or answers questions based on those retrieved documents. |

**The advantage is:**
- **ChromaDB** does the *search* step (finds similar content).
- **OpenAI** does the *reasoning* step (understands context, answers questions smartly, stitches ideas together).

Chroma gives you **pieces** that are relevant, but OpenAI can **interpret**, **summarize**, or **formulate a proper answer** based on them — even if the pieces are incomplete, partial, or noisy.

---
Here's a quick metaphor:  
- ChromaDB is like a **librarian** who gives you the 5 most relevant books.
- OpenAI is like a **research assistant** who **reads** those books and writes you a **coherent essay** answering your question.

---
**Why not just use ChromaDB output directly?**
- **Because retrieved chunks may be partial or need reasoning.**  
  Example: Chroma may retrieve 3 paragraphs about "how to cook pasta", but the actual *answer* (e.g., combining steps correctly) requires reasoning across those.
- **Summarization and rephrasing.**  
  Raw data might be too verbose, too technical, or not phrased well for the user's question.
- **Answering specific questions.**  
  Sometimes the user asks a *new synthesis* (e.g., “what’s the key difference between A and B?”), not just a retrieval of existing text.

---

**In short:**  
👉 Chroma finds **what's relevant**.  
👉 OpenAI explains **why it matters** (and builds a **smart answer** from it).

---
Would you like me to sketch a simple example pipeline showing how they interact? 🚀 (might make it even clearer!)

## Question:
So, the LLM will use the external Knowledge also to fetch the coherent answer or just the info being saved in Vector store in the form of Embeddings ?

Awesome — you're asking exactly the right next question.

**Answer:**  
👉 If you **only** pass the ChromaDB results (retrieved documents) as the **context** to the LLM **without giving it external knowledge**,  
then **the LLM is restricted to answering *only* based on that provided context**.

**It won't use its own general knowledge** unless you allow or instruct it to.

---

🔵 **More precisely:**  
When you design your prompt to the LLM after fetching from ChromaDB, you usually say something like:
> "Based only on the following documents, answer the question."

This **forces the LLM** to act as if it only knows the provided info (like a closed-book test).

---
🔵 **Alternatively**, if you **don't restrict** the LLM, it might mix:
- knowledge from the ChromaDB documents, **and**
- its own internal trained knowledge (from 2023, in GPT-4-Turbo for example).

This sometimes makes it "hallucinate" (invent) facts if your documents don't fully cover the answer.

---

| Case | How LLM behaves |
|:---|:---|
| **Documents only** (typical RAG) | LLM reads only the ChromaDB-retrieved context and generates an answer based strictly on it. |
| **Free-form** (default chat) | LLM can guess, add its own knowledge, or hallucinate if the context is missing something. |

---
✅ So in a **serious RAG system** (Retrieval Augmented Generation),  
you usually *want* to tell the LLM:
- **“Only use the provided context.”**  
- **“If you don't know, say you don't know.”**  

To avoid hallucination.


## Langchain

In [6]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage 

In [2]:
load_dotenv()

True

In [9]:
openai_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(api_key=openai_key, model="gpt-4o-mini")

In [13]:
messages = [
    SystemMessage("Translate the following from English into Telugu"),
    HumanMessage("hi!"),
]

In [14]:
model.invoke(messages)

AIMessage(content='హాయ్!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 20, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BQpTzSGjWnlW01DquwJ17FS0vL23f', 'finish_reason': 'stop', 'logprobs': None}, id='run-d9c66ea4-e715-4d7c-b8ac-5e957ec96b5e-0', usage_metadata={'input_tokens': 20, 'output_tokens': 5, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
model.invoke('hello')
model.invoke([{"role":"user","content":'Hello'}])
model.invoke([HumanMessage("hello")])

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BQpQIjHOft2sp4YlAxglDPhyNv47a', 'finish_reason': 'stop', 'logprobs': None}, id='run-c25169c5-255d-49b4-9922-86f3464b4247-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [29]:
import os

from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

In [35]:
from langchain.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    path="C:\\Users\\somashekar.inguva\\OneDrive - Telstra\\Documents\\GenAI_Projects\\vector-databases-course\\data\\new_articles",
    glob="*.txt",
    loader_cls=TextLoader,
    encoding = 'utf-8'
)
document = loader.load()
document


TypeError: DirectoryLoader.__init__() got an unexpected keyword argument 'encoding'

In [33]:
loader = DirectoryLoader(
    path=r"C:\Users\somashekar.inguva\OneDrive - Telstra\Documents\GenAI_Projects\vector-databases-course\data\new_articles",
    glob="*.txt",
    loader_cls=lambda path: TextLoader(path, encoding='utf-8')
)
document = loader.load()


In [34]:
document

[Document(metadata={'source': 'C:\\Users\\somashekar.inguva\\OneDrive - Telstra\\Documents\\GenAI_Projects\\vector-databases-course\\data\\new_articles\\05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt'}, page_content='Signaling that investments in the supply chain sector remain robust, Pando, a startup developing fulfillment management technologies, today announced that it raised $30 million in a Series B round, bringing its total raised to $45 million.\n\nIron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.\n\n“We will not expand into new industries or adjacent product areas,” he told TechCrunch in an email interview. “Great talent is the foundation of the business — we will continue to augment our teams at all levels of t

In [40]:
len(document)

21

In [41]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n,\n","\n"],
    chunk_size = 1000,
    chunk_overlap = 20
)

In [42]:
documents = text_splitter.split_documents(document)
print(f"num of docs:{len(documents)}")

num of docs:223


In [43]:
documents

[Document(metadata={'source': 'C:\\Users\\somashekar.inguva\\OneDrive - Telstra\\Documents\\GenAI_Projects\\vector-databases-course\\data\\new_articles\\05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt'}, page_content='Signaling that investments in the supply chain sector remain robust, Pando, a startup developing fulfillment management technologies, today announced that it raised $30 million in a Series B round, bringing its total raised to $45 million.\n\nIron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.\n\n“We will not expand into new industries or adjacent product areas,” he told TechCrunch in an email interview. “Great talent is the foundation of the business — we will continue to augment our teams at all levels of t

In [44]:
embedding = OpenAIEmbeddings(api_key=openai_key,model="text-embedding-3-small")

In [47]:
persistdir = "./db/chroma_db_real_world1"

In [48]:
vectorDB = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persistdir
)

In [49]:
retriever = vectorDB.as_retriever()

In [51]:
res_docs = retriever.invoke("how much did microsoft raise?",k=2)
print(res_docs)

[Document(metadata={'source': 'C:\\Users\\somashekar.inguva\\OneDrive - Telstra\\Documents\\GenAI_Projects\\vector-databases-course\\data\\new_articles\\05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}, page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enter

In [73]:
res_docs = retriever.invoke("who is Dhoni",k=2)
print(res_docs)

[Document(metadata={'source': 'C:\\Users\\somashekar.inguva\\OneDrive - Telstra\\Documents\\GenAI_Projects\\vector-databases-course\\data\\new_articles\\05-07-3one4-capital-driven-by-contrarian-bets-raises-200-million-new-fund.txt'}, page_content='Pai, pictured above, asserts that there is ample room for more Indian companies to pursue IPOs, as the nation’s IPO market has proven successful and well-regulated for institutional investors. He anticipates a transformation in India’s stock index, with an increasing number of tech companies, apps, services, fintech, and payment solutions becoming part of the index.\n\nDespite this, Pai acknowledges that the Indian market has yet to fully realize its potential for mergers and acquisitions. Although there has been growth in M&A activity—increasing three to four times in the past five years—it remains below expectations. For the Indian market to flourish, Pai emphasizes the need for a more robust M&A landscape.'), Document(metadata={'source': '

In [65]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [66]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [67]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [68]:
question_answer_chain = create_stuff_documents_chain(
    llm=model,
    prompt=prompt,
)

In [69]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [71]:
resp = rag_chain.invoke({"input": "talk about databricks news"})
res = resp["answer"]
res

"Databricks recently announced its acquisition of Okera, a data governance platform focused on AI. The acquisition aims to integrate Okera's advanced governance technologies into Databricks' Unity Catalog, enhancing data management and compliance. This move aligns with the growing need for efficient governance solutions as data complexity increases across organizations."

In [70]:
resp = rag_chain.invoke({"input": "who is Sachin Tendulkar"})
res = resp["answer"]
res

'Sachin Tendulkar is a former Indian cricketer widely regarded as one of the greatest batsmen in the history of cricket. He made his international debut at the age of 16 and has numerous records to his name, including being the first player to score 100 international centuries. Tendulkar is also known for his significant contributions to Indian cricket and has received numerous awards for his achievements both on and off the field.'

In [72]:
resp = rag_chain.invoke({"input": "who is Dhoni"})
res = resp["answer"]
res

'Dhoni refers to Mahendra Singh Dhoni, a former Indian cricketer and captain of the Indian national team. He is widely regarded as one of the greatest wicket-keeper batsmen and captains in the history of cricket, known for his calm demeanor and strategic acumen. Under his leadership, India won several major tournaments, including the ICC T20 World Cup in 2007 and the ICC Cricket World Cup in 2011.'

## Question

So, Using Lanchain and rag we can retrieve the info not related to the docs chunks..right ?

Yes, using LangChain with RAG (Retrieval-Augmented Generation) allows you to retrieve information that isn't strictly from document chunks. Here’s a breakdown:

Understanding RAG: In RAG, the process combines retrieval of information from a database with generation capabilities of language models. This means you can get information that is pertinent but not limited to the chunks of documents.

Retrieving Information: With LangChain, you can efficiently query a database and obtain relevant information that may extend beyond just the chunks. The model can leverage other sources of knowledge or metadata to generate responses.

Inference: You initiate a query, and based on the relevant context from the retrieved documents (which may include metadata), the model generates a well-structured answer that addresses your question directly.

This method enhances the ability to provide direct answers rather than being constrained only to the available chunks.